In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [48]:
ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()
data = yf.download(ticker, period="1y")  # Fetch stock data
print(data.head())  # Debugging: Check if data is fetched

Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA):  TSLA


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2024-03-20  175.660004  176.250000  170.820007  173.000000   83846700
2024-03-21  172.820007  178.179993  171.800003  176.389999   73178000
2024-03-22  170.830002  171.199997  166.300003  166.690002   75454700
2024-03-25  172.630005  175.240005  168.729996  168.759995   74228600
2024-03-26  177.669998  184.250000  177.380005  178.580002  113186200


In [28]:
def get_rsi(data, period=14):
    """

    Calculates the Relative Strength Index (RSI) for a given stock data.
    """
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Prevent division by zero
    rs = avg_gain / avg_loss.replace(0, np.nan)  # Replace 0 with NaN to avoid division by zero
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


In [29]:
def get_macd(data, short_window=12, long_window=26, signal_window=9):

    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

In [30]:
def get_bollinger_bands(data, window=20, num_std=2):
   
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bandwidth = (upper_band - lower_band) / rolling_mean
    return upper_band, lower_band, bandwidth


In [31]:
def get_momentum(data, period=14):
 
    return data['Close'].diff(period)


In [32]:
def get_volatility(data, period=14):
  
    return data['Close'].pct_change().rolling(window=period).std()


In [33]:
def get_stock_data(ticker):
    
    data = yf.download(ticker, period="1y")  # Download 1 year of data
    return data

In [34]:
def prepare_data(data):
    """
    Prepares the data for machine learning by adding features and labels.
    """
    
    # Calculate price changes FIRST
    data['Price_Change'] = data['Close'].pct_change()

    # Add RSI
    data['RSI'] = get_rsi(data)

    # Check if RSI was created successfully (case insensitive)
    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    # Add Moving Averages
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    # Add MACD
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    # Add Bollinger Bands
    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    # Add Momentum
    data['Momentum'] = get_momentum(data)

    # Add Volatility
    data['Volatility'] = get_volatility(data)

   # Add Volume Change
       # Add Volume Change
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    # Define buy/sell signals with NaN handling
    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False)  # Fix: Assign the result back to buy_condition
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False)  # Fix: Assign the result back to sell_condition

    # Set the Signal column based on conditions
    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1  # Buy signal
    data.loc[sell_condition, 'Signal'] = -1  # Sell signal

    return data

In [35]:
def prepare_data(data):
    """
    Prepares the data for machine learning by adding features and labels.
    """
    # Calculate price changes FIRST
    data['Price_Change'] = data['Close'].pct_change()

    # Add RSI
    data['RSI'] = get_rsi(data)

    # Check if RSI was created successfully (case insensitive)
    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    # Add Moving Averages
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    # Add MACD
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    # Add Bollinger Bands
    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    # Add Momentum
    data['Momentum'] = get_momentum(data)

    # Add Volatility
    data['Volatility'] = get_volatility(data)

    # Add Volume Change
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    # Define buy/sell signals with NaN handling
    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False)  # Fix: Assign the result back to buy_condition
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False)  # Fix: Assign the result back to sell_condition

    # Set the Signal column based on conditions
    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1  # Buy signal
    data.loc[sell_condition, 'Signal'] = -1  # Sell signal

    # Drop rows with NaN values (due to feature calculations)
    data.dropna(inplace=True)
    return data

In [36]:
def train_model(data):
    # Define feature columns
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    # Ensure features exist in the data
    missing_cols = [col for col in features if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in data: {missing_cols}")

    # Extract features and target variable
    X = data[features]
    y = data.get('Signal')  # Avoid KeyError

    if y is None:
        raise ValueError("Target variable 'Signal' is missing from the dataset.")

    # Convert to DataFrame to avoid warnings
    y = y.to_frame()

    # Handle NaN values: Fill with column mean
    imputer = SimpleImputer(strategy="mean")
    X = pd.DataFrame(imputer.fit_transform(X), columns=features, index=data.index)

    # Remove NaNs from y
    y.dropna(inplace=True)

    # Ensure X and y stay aligned
    X = X.loc[y.index]

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train.values.ravel())  # Convert y_train to a 1D array

    return model

In [46]:
def generate_alerts(data, model):
    # Features
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    # Predict signals using the trained model
    data['Predicted_Signal'] = model.predict(data[features])

    # Generate alerts
    alerts = []
    for i in range(len(data)):  # <-- Fixed indentation here
        if data['Predicted_Signal'].iloc[i] == 1:
            alerts.append(("Buy", data.index[i], data['Close'].iloc[i]))
        elif data['Predicted_Signal'].iloc[i] == -1:
            alerts.append(("Sell", data.index[i], data['Close'].iloc[i]))
        else:
            alerts.append(("Hold", data.index[i], data['Close'].iloc[i]))

    return alerts

In [47]:
def analyze_stock(ticker):
    """
    Analyzes a stock and provides buy/sell/hold alerts.
    """
    # Fetch and prepare data
    data = get_stock_data(ticker)
    data = prepare_data(data)

    # Train the model
    model = train_model(data)

    # Generate alerts
    alerts = generate_alerts(data, model)

    # Print alerts
    print(f"\nAlerts for {ticker}:")
    for alert in alerts:
        print(f"{alert[0]} Signal at {alert[1]}: Price: {alert[2]}")

if __name__ == "__main__":
    # Input stock ticker from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()

    # Analyze the stock
    analyze_stock(ticker)

Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA):  TSLA


[*********************100%***********************]  1 of 1 completed


Alerts for TSLA:
Hold Signal at 2024-05-30 00:00:00: Price: Ticker
TSLA    178.789993
Name: 2024-05-30 00:00:00, dtype: float64
Hold Signal at 2024-05-31 00:00:00: Price: Ticker
TSLA    178.080002
Name: 2024-05-31 00:00:00, dtype: float64
Hold Signal at 2024-06-03 00:00:00: Price: Ticker
TSLA    176.289993
Name: 2024-06-03 00:00:00, dtype: float64
Hold Signal at 2024-06-04 00:00:00: Price: Ticker
TSLA    174.770004
Name: 2024-06-04 00:00:00, dtype: float64
Hold Signal at 2024-06-05 00:00:00: Price: Ticker
TSLA    175.0
Name: 2024-06-05 00:00:00, dtype: float64
Hold Signal at 2024-06-06 00:00:00: Price: Ticker
TSLA    177.940002
Name: 2024-06-06 00:00:00, dtype: float64
Hold Signal at 2024-06-07 00:00:00: Price: Ticker
TSLA    177.479996
Name: 2024-06-07 00:00:00, dtype: float64
Hold Signal at 2024-06-10 00:00:00: Price: Ticker
TSLA    173.789993
Name: 2024-06-10 00:00:00, dtype: float64
Hold Signal at 2024-06-11 00:00:00: Price: Ticker
TSLA    170.660004
Name: 2024-06-11 00:00:00, dty

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def get_rsi(data, period=14):
    """Calculates the Relative Strength Index (RSI)."""
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def get_macd(data, short_window=12, long_window=26, signal_window=9):
    """Calculates MACD indicators."""
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal, macd - signal

def get_bollinger_bands(data, window=20, num_std=2):
    """Calculates Bollinger Bands."""
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band, (upper_band - lower_band) / rolling_mean

def prepare_data(data):
    """Prepares stock data with technical indicators."""
    data['RSI'] = get_rsi(data)
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    data['Momentum'] = data['Close'].diff(14)
    data['Volatility'] = data['Close'].pct_change().rolling(14).std()
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    data['Price_Change'] = data['Close'].pct_change()
    data.dropna(inplace=True)
    return data

def train_model(data):
    """Trains a model to predict stock prices."""
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']

    X = data[features]
    y = data['Close']

    # Handle missing values
    imputer = SimpleImputer(strategy="mean")
    X = pd.DataFrame(imputer.fit_transform(X), columns=features, index=data.index)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Scale features while keeping the DataFrame structure
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=features, index=X_train.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=features, index=X_test.index)

    # Train a Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    return model, scaler, X_test_scaled, y_test, X_test.index

def forecast_prices(data, model, scaler):
    """Forecasts future stock prices."""
    future_data = data[-30:].copy()  # Use last 30 days as reference
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']

    future_features = future_data[features]
    future_scaled = scaler.transform(future_features)
    future_predictions = model.predict(future_scaled)

    future_dates = pd.date_range(start=data.index[-1], periods=30, freq='B')  # Business days
    forecast_df = pd.DataFrame({'Date': future_dates, 'Predicted_Close': future_predictions})
    
    return forecast_df

def plot_predictions(data, y_test, y_pred, forecast_df):
    """Plots actual vs. predicted prices."""
    plt.figure(figsize=(12, 6))

    # Actual vs. predicted
    plt.plot(data.index, data['Close'], label="Actual Prices", color='blue', alpha=0.6)
    plt.plot(y_test.index, y_pred, label="Predicted Prices", color='red', linestyle="dashed")

    # Future forecast
    plt.plot(forecast_df['Date'], forecast_df['Predicted_Close'], label="Forecasted Prices", color='green', linestyle="dotted")

    plt.xlabel("Date")
    plt.ylabel("Stock Price")
    plt.legend()
    plt.title("Stock Price Prediction & Forecast")
    plt.show()

def analyze_stock(ticker):
    """Analyzes a stock and forecasts its future prices."""
    data = yf.download(ticker, period="1y")
    data = prepare_data(data)

    model, scaler, X_test_scaled, y_test, test_index = train_model(data)

    # Predict on test data
    y_pred = model.predict(X_test_scaled)

    # Forecast future prices
    forecast_df = forecast_prices(data, model, scaler)

    # Plot results
    plot_predictions(data, y_test, y_pred, forecast_df)

if __name__ == "__main__":
    ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()
    analyze_stock(ticker)
